In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedKFold
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import xgboost as xgb
import lightgbm as lgb
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

## Loading data

In [4]:
x_train = pd.read_csv('x_train.csv', index_col='ID')
y_train = pd.read_csv('y_train.csv', index_col='ID')
train = pd.concat([x_train, y_train], axis=1)
test = pd.read_csv('x_test.csv', index_col='ID')
# Ensure proper time ordering within each stock
train = train.sort_values(['STOCK', 'DATE']).reset_index()
test  = test.sort_values(['STOCK', 'DATE']).reset_index()
train.head()

,ID,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,...,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20,RET
0,2377,1,0,37,12,5,94,-0.005967,0.136699,0.009031,...,-0.493354,-0.007660,-0.585497,-0.001063,-0.351363,0.005127,-0.324675,-0.019275,-0.291751,False
1,5198,4,0,37,12,5,94,0.001348,-0.269520,0.011100,...,-0.313575,0.007867,0.071338,0.007733,-0.405243,-0.003276,-0.424336,-0.010489,-0.050591,False
2,8017,5,0,37,12,5,94,-0.014405,0.192655,0.003614,...,-0.367499,-0.005843,-0.405562,0.002930,-0.315935,0.010462,-0.474957,-0.003541,-0.260130,True
3,20826,11,0,37,12,5,94,0.008938,0.430916,0.002662,...,0.023598,0.011266,0.079711,0.019038,-0.230167,-0.000287,-0.312123,0.008682,-0.226628,True
4,33843,21,0,37,12,5,94,-0.006523,-0.060371,-0.007632,...,-0.337686,-0.007224,-0.161117,-0.001461,-0.095494,0.012667,0.471895,-0.038752,1.532045,False


# Feature Engineering

## Experiment with new features
From v2, we know:
- VOLUME_1 is #1 for both models → Confirms volume is critical
- vol_5d is #2 for both → Volatility is key predictor
- Rank-based features appear in top 20 for both models
Differences between models:
- XGBoost loves raw features: VOLUME_1 (340), vol_5d (251)
- LightGBM prefers engineered features: RET_1_vs_sector_med (0.0317), RET_1_cs_zscore (0.0302)



In [5]:
def create_enhanced_cross_sectional_features(df):
    """Combined v2 cross-sectional and lucabri technical features"""
    df_new = df.copy()
    
    print("  Creating enhanced rank-based features...")
    
    if 'DATE' not in df_new.columns:
        return df_new
    
    # 1. PERCENTILE RANKS
    for col in [f'RET_{i}' for i in range(1, 21)]:
        if col in df_new.columns:
            df_new[f'{col}_pctrank'] = df_new.groupby('DATE')[col].rank(pct=True)
    
    # 2. SECTOR-RELATIVE RANKS
    if 'SECTOR' in df_new.columns:
        for col in [f'RET_{i}' for i in range(1, 11)]:
            if col in df_new.columns:
                df_new[f'{col}_sector_pctrank'] = df_new.groupby(['DATE', 'SECTOR'])[col].rank(pct=True)
    
    # 3. VOLUME RANKS
    for col in [f'VOLUME_{i}' for i in range(1, 11) if f'VOLUME_{i}' in df_new.columns]:
        df_new[f'{col}_pctrank'] = df_new.groupby('DATE')[col].rank(pct=True)
    
    # 4. CROSS-SECTIONAL Z-SCORES
    for col in [f'RET_{i}' for i in range(1, 11)]:
        if col in df_new.columns:
            df_new[f'{col}_cs_zscore'] = df_new.groupby('DATE')[col].transform(
                lambda x: (x - x.mean()) / (x.std() + 1e-8)
            )
    
    # 5. RELATIVE TO SECTOR MEDIAN
    if 'SECTOR' in df_new.columns:
        for col in [f'RET_{i}' for i in range(1, 6)]:
            if col in df_new.columns:
                sector_median = df_new.groupby(['DATE', 'SECTOR'])[col].transform('median')
                df_new[f'{col}_vs_sector_med'] = df_new[col] - sector_median
    
    # 6. COMPOSITE MOMENTUM RANKS
    df_new['momentum_1_5'] = df_new[[f'RET_{i}' for i in range(1, 6)]].mean(axis=1)
    df_new['momentum_1_5_pctrank'] = df_new.groupby('DATE')['momentum_1_5'].rank(pct=True)
    
    df_new['momentum_6_20'] = df_new[[f'RET_{i}' for i in range(6, 21)]].mean(axis=1)
    df_new['momentum_6_20_pctrank'] = df_new.groupby('DATE')['momentum_6_20'].rank(pct=True)
    
    # 7. VOLATILITY RANKS
    df_new['vol_5d'] = df_new[[f'RET_{i}' for i in range(1, 6)]].std(axis=1)
    df_new['vol_5d_pctrank'] = df_new.groupby('DATE')['vol_5d'].rank(pct=True)
    
    # 8. RETURN-TO-VOLATILITY RATIO
    df_new['ret_vol_ratio'] = df_new['momentum_1_5'] / (df_new['vol_5d'] + 1e-8)
    df_new['ret_vol_ratio_pctrank'] = df_new.groupby('DATE')['ret_vol_ratio'].rank(pct=True)
    
    print("  Adding technical indicators...")
    
    # 9. SECTOR AGGREGATES
    if 'SECTOR' in df_new.columns:
        for shift in [1, 2, 3, 4]:
            feat = f'RET_{shift}'
            if feat in df_new.columns:
                df_new[f'{feat}_sector_mean'] = df_new.groupby(['SECTOR', 'DATE'])[feat].transform('mean')
    
    # 10. WEEKLY STATISTICS
    for week in range(1, 5):
        ret_cols = [f'RET_{(week-1)*5 + d}' for d in range(1, 6) if f'RET_{(week-1)*5 + d}' in df_new.columns]
        vol_cols = [f'VOLUME_{(week-1)*5 + d}' for d in range(1, 6) if f'VOLUME_{(week-1)*5 + d}' in df_new.columns]
        
        if ret_cols:
            df_new[f'ret_week{week}_mean'] = df_new[ret_cols].mean(axis=1)
            df_new[f'ret_week{week}_std'] = df_new[ret_cols].std(axis=1)
        if vol_cols:
            df_new[f'vol_week{week}_mean'] = df_new[vol_cols].mean(axis=1)
    
    # 11. RSI (RELATIVE STRENGTH INDEX)
    if 'SECTOR' in df_new.columns:
        ret_20 = [f'RET_{i}' for i in range(1, 21) if f'RET_{i}' in df_new.columns]
        if ret_20:
            gains = df_new.groupby(['SECTOR', 'DATE'])[ret_20].mean().agg(lambda x: x[x>0].mean(), axis=1)
            losses = df_new.groupby(['SECTOR', 'DATE'])[ret_20].mean().agg(lambda x: x[x<0].mean(), axis=1).abs()
            rs = gains / losses
            rsi = 100 - (100 / (1 + rs))
            df_new = df_new.join(rsi.to_frame('rsi_sector'), on=['SECTOR', 'DATE'], how='left')
    
    # 12. ADL (ADVANCE-DECLINE LINE)
    if 'SECTOR' in df_new.columns:
        ret_5 = [f'RET_{i}' for i in range(1, 6) if f'RET_{i}' in df_new.columns]
        if ret_5:
            adl = df_new.groupby(['SECTOR', 'DATE'])[ret_5].apply(
                lambda x: (x > 0).sum().sum() - (x < 0).sum().sum()
            )
            df_new = df_new.join(adl.to_frame('adl_sector'), on=['SECTOR', 'DATE'], how='left')
    
    return df_new


# Apply enhanced features
print("\nApplying to train...")
train_eng = create_enhanced_cross_sectional_features(train)
print(f"✓ Train shape: {train_eng.shape}")

print("\nApplying to test...")
test_eng = create_enhanced_cross_sectional_features(test)
print(f"✓ Test shape: {test_eng.shape}")


Applying to train...
  Creating enhanced rank-based features...
  Adding technical indicators...
✓ Train shape: (418595, 129)

Applying to test...
  Creating enhanced rank-based features...
  Adding technical indicators...
✓ Test shape: (198429, 128)


## Baseline model training and feature importance analysis

In [12]:
imputer = SimpleImputer(strategy='median')

categorical_cols = ['DATE', 'STOCK', 'INDUSTRY', 'INDUSTRY_GROUP', 'SECTOR', 'SUB_INDUSTRY']
exclude_cols = ['ID', 'RET'] + categorical_cols
feature_cols = [col for col in train_eng.columns if col not in exclude_cols]

X_train = train_eng[feature_cols].copy()
y_train = train_eng['RET'].astype(int)

X_train_imp = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test_imp = pd.DataFrame(
    imputer.transform(test_eng[feature_cols]),
    columns=test_eng[feature_cols].columns,
    index=test_eng.index
)


In [13]:

# Baseline RF for feature importance
print("\nTraining baseline RF...")
rf_baseline = RandomForestClassifier(
    n_estimators=300,
    max_depth=6,
    min_samples_split=20,
    random_state=42,
    n_jobs=-1
)

rf_baseline.fit(X_train_imp, y_train)

# Feature importance
feat_imp = pd.DataFrame({
    'feature': X_train_imp.columns,
    'importance': rf_baseline.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 50 features:")
print(feat_imp.head(50))


Training baseline RF...

Top 50 features:
                  feature  importance
106     RET_4_sector_mean    0.067508
105     RET_3_sector_mean    0.059645
103     RET_1_sector_mean    0.050761
119            rsi_sector    0.038356
32                 RET_17    0.033857
104     RET_2_sector_mean    0.033352
120            adl_sector    0.031354
99                 vol_5d    0.029306
108         ret_week1_std    0.026875
80        RET_1_cs_zscore    0.024932
1                VOLUME_1    0.022914
0                   RET_1    0.021276
70       VOLUME_1_pctrank    0.020280
100        vol_5d_pctrank    0.020194
90    RET_1_vs_sector_med    0.018039
116        ret_week4_mean    0.016129
40          RET_1_pctrank    0.014922
97          momentum_6_20    0.013062
111         ret_week2_std    0.011968
114         ret_week3_std    0.011215
57         RET_18_pctrank    0.011185
117         ret_week4_std    0.011098
4                   RET_3    0.010809
86        RET_7_cs_zscore    0.010806
110    

# Model Training

In [14]:
def train_model_cv(X, y, dates, model_name='xgb', n_folds=5):
    gkf = GroupKFold(n_splits=n_folds)
    results = []
    
    for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups=dates)):
        X_train_fold = X.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_train_fold = y.iloc[train_idx]
        y_val_fold = y.iloc[val_idx]
        
        # Train model
        if model_name == 'xgb':
            model = xgb.XGBClassifier(
                n_estimators=300,
                max_depth=5,
                learning_rate=0.03,
                subsample=0.7,
                colsample_bytree=0.6,
                reg_alpha=0.8,
                reg_lambda=2.5,
                random_state=42 + fold,
                n_jobs=-1,
                use_label_encoder=False
            )
        elif model_name == 'lgb':
            model = lgb.LGBMClassifier(
                n_estimators=300,
                max_depth=5,
                learning_rate=0.02,
                subsample=0.7,
                colsample_bytree=0.7,
                min_child_samples=30,
                reg_alpha=1.5,
                reg_lambda=3.5,
                random_state=42 + fold,
                n_jobs=-1,
                verbose=-1
            )
        elif model_name == 'rf':
            model = RandomForestClassifier(
                n_estimators=500,
                max_depth=6,
                min_samples_split=20,
                min_samples_leaf=10,
                max_features='sqrt',
                random_state=42 + fold,
                n_jobs=-1
            )
        
        model.fit(X_train_fold, y_train_fold)
        
        # Predict with standard threshold (0.5)
        val_proba = model.predict_proba(X_val_fold)[:, 1]
        y_pred = (val_proba > 0.5).astype(int)
        
        # Calculate metrics
        acc = accuracy_score(y_val_fold, y_pred)
        auc = roc_auc_score(y_val_fold, val_proba)
        
        results.append({
            'Fold': fold+1,
            'Accuracy': acc,
            'AUC': auc,
            'Model': model,
            'Feature_Names': X.columns.tolist()
        })
        
        print(f"Fold {fold+1}: Accuracy={acc:.4f}, AUC={auc:.4f}")
    
    return results

top_features = feat_imp.head(50)['feature'].tolist()
X_train_top = X_train_imp[top_features]
X_test_top = X_test_imp[top_features]

print(f"\nReduced from {X_train_imp.shape[1]} to {X_train_top.shape[1]} features")

# Now train with selected features
print("\n1. Training XGBoost...")
xgb_results = train_model_cv(X_train_top, y_train, train_eng['DATE'].values, 'xgb')

print("\n2. Training LightGBM...")
lgb_results = train_model_cv(X_train_top, y_train, train_eng['DATE'].values, 'lgb')

print("\n3. Training RandomForest...")
rf_results = train_model_cv(X_train_top, y_train, train_eng['DATE'].values, 'rf')
# ============================================================================
# MODEL COMPARISON SUMMARY
# ============================================================================
print("\n" + "="*80)
print("MODEL COMPARISON SUMMARY")
print("="*80)

for model_name, results in [('XGBoost', xgb_results), ('LightGBM', lgb_results), ('RandomForest', rf_results)]:
    acc_mean = np.mean([r['Accuracy'] for r in results])
    acc_std = np.std([r['Accuracy'] for r in results])
    auc_mean = np.mean([r['AUC'] for r in results])
    auc_std = np.std([r['AUC'] for r in results])
    
    print(f"\n{model_name}:")
    print(f"  Mean Accuracy:  {acc_mean:.4f} (±{acc_std:.4f})")
    print(f"  Mean AUC:       {auc_mean:.4f} (±{auc_std:.4f})")



Reduced from 121 to 50 features

1. Training XGBoost...
Fold 1: Accuracy=0.5170, AUC=0.5228
Fold 2: Accuracy=0.5167, AUC=0.5204
Fold 3: Accuracy=0.5163, AUC=0.5208
Fold 4: Accuracy=0.5147, AUC=0.5218
Fold 5: Accuracy=0.5200, AUC=0.5287

2. Training LightGBM...
Fold 1: Accuracy=0.5102, AUC=0.5144
Fold 2: Accuracy=0.5181, AUC=0.5227
Fold 3: Accuracy=0.5183, AUC=0.5247
Fold 4: Accuracy=0.5161, AUC=0.5206
Fold 5: Accuracy=0.5180, AUC=0.5235

3. Training RandomForest...
Fold 1: Accuracy=0.5068, AUC=0.5124
Fold 2: Accuracy=0.5167, AUC=0.5243
Fold 3: Accuracy=0.5165, AUC=0.5241
Fold 4: Accuracy=0.5148, AUC=0.5218
Fold 5: Accuracy=0.5166, AUC=0.5252

MODEL COMPARISON SUMMARY

XGBoost:
  Mean Accuracy:  0.5169 (±0.0017)
  Mean AUC:       0.5229 (±0.0030)

LightGBM:
  Mean Accuracy:  0.5161 (±0.0031)
  Mean AUC:       0.5212 (±0.0036)

RandomForest:
  Mean Accuracy:  0.5143 (±0.0038)
  Mean AUC:       0.5216 (±0.0047)
